In [ ]:
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import numpy as np 
from scipy.optimize import minimize_scalar

if str(Path().absolute().parents[1]) not in sys.path:
    sys.path.insert(0, str(Path().absolute().parents[1]))
from cv_prince.chap_08_linear_regression.ml_linear_regression import LinearRegression
from cv_prince.chap_08_linear_regression.bayes_linear_regression import BayesLinearRegression
from cv_prince.chap_08_linear_regression.non_linear_regression import NonLinearRegression
from cv_prince.chap_08_linear_regression.gauss_process_regression import GaussProcessRegression
from cv_prince.chap_08_linear_regression.sparse_linear_regression import SparseLinearRegression
from cv_prince.chap_08_linear_regression.relevance_vector_regression import RelevanceVectorRegression

for path in sys.path:
    print(path)

In [ ]:
def create_noisy_linear(direction,
    offset: np.ndarray,
    nsamples: int,
    xrange: np.ndarray,
    inlier_noise_std: float,
    outlier_rate: float,
    rng: np.random.Generator
) -> tuple[np.ndarray, np.ndarray]:
    xmin, xmax = xrange

    samples_x = rng.uniform(xmin, xmax, nsamples)
    samples_x.sort()

    samples_y = direction * samples_x + offset + rng.normal(0, inlier_noise_std, nsamples)

    outlier_perturb = rng.uniform(200, 500, nsamples) * rng.choice([-1, 1], nsamples)
    samples_y += rng.binomial(1, outlier_rate, nsamples) * outlier_perturb

    return samples_x, samples_y

### Maximum Likelihood Linear Regression

In [ ]:
nsamples = 100
outlier_rate = 0.0
inlier_noise_std = 50.0

direction = 2.0
offset = 3.0

xmin = -100
xmax = 100

rng = np.random.default_rng(seed=12345)

samples_x, samples_y = create_noisy_linear(
    direction,
    offset,
    nsamples,
    [xmin, xmax],
    inlier_noise_std,
    outlier_rate,
    rng
)

In [ ]:
linreg = LinearRegression()
linreg.fit(samples_x, samples_y)

xrange = np.array([samples_x.min(), samples_x.max()])
yrange = np.array([samples_y.min(), samples_y.max()])

grid_size = 1000
grid_x, grid_y = np.meshgrid(
    np.linspace(xrange[0] , xrange[1] , grid_size), 
    np.linspace(yrange[0] , yrange[1] , grid_size),
    indexing="xy"
)

likelihood = linreg.likelihood(grid_x.flatten(), grid_y.flatten())
likelihood = likelihood.reshape((grid_size, grid_size))

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.scatter(samples_x, samples_y, color="b")
ax.plot(xrange, linreg.predict(xrange), color="red")
ax.imshow(
    likelihood, 
    extent=xrange.tolist() + yrange.tolist(),  
    origin='lower', 
    aspect='auto',
    cmap="hot"
)
fig.tight_layout()

### Bayesian Linear Regression

In [ ]:
nsamples = 20
outlier_rate = 0.1
inlier_noise_std = 10.0

direction = 2.0
offset = 3.0

xrange = np.array([-6, 6])
samples_x = np.array([-3.9, -0.86, -0.35, -0.24, 0.1, 0.9, 3])
samples_y = np.array([4, 2, 0.5, -2, 0.9, 0.6, 0.25])

In [ ]:
ml_linreg = LinearRegression()
ml_linreg.fit(samples_x, samples_y)

In [ ]:
sigma_p = 100
bayes_linreg = BayesLinearRegression(sigma_prior=sigma_p)
bayes_linreg.fit(samples_x, samples_y)

In [ ]:
xrange = np.array([-6, 6])
yrange = np.array([samples_y.min()-2, samples_y.max()+2])

grid_size = 1000
grid_x, grid_y = np.meshgrid(
    np.linspace(xrange[0] , xrange[1] , grid_size), 
    np.linspace(yrange[0] , yrange[1] , grid_size),
    indexing="xy"
)

bayes_likelihood = bayes_linreg.likelihood(grid_x.flatten(), grid_y.flatten())
bayes_likelihood = bayes_likelihood.reshape((grid_size, grid_size))

ml_likelihood = ml_linreg.likelihood(grid_x.flatten(), grid_y.flatten())
ml_likelihood = ml_likelihood.reshape((grid_size, grid_size))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=[15, 7])

ax[0].scatter(samples_x, samples_y, color="b")
ax[0].plot(xrange, ml_linreg.predict(xrange), color="red")
ax[0].imshow(
    ml_likelihood, 
    extent=[xrange[0], xrange[1], samples_y.min()-2, samples_y.max()+2],
    origin='lower', 
    aspect='auto',
    cmap="hot"
)
ax[0].set_title("ML")

ax[1].scatter(samples_x, samples_y, color="b")
ax[1].plot(xrange, bayes_linreg.predict(xrange), color="red")
ax[1].imshow(
    bayes_likelihood, 
    extent=[xrange[0], xrange[1], samples_y.min()-2, samples_y.max()+2],  
    origin='lower', 
    aspect='auto',
    cmap="hot"
)
ax[1].set_title("Bayes")

fig.tight_layout()

### Non Linear Regression

In [ ]:
gt_non_lin_func = lambda x: 1.5 * np.sin(x ** 2) + np.cos(x ** 3)

xmin, xmax = 0, 3.05
xrange = np.linspace(xmin, xmax, 1000)

rng = np.random.default_rng(seed=12345)
nsamples = 50
samples_x = rng.uniform(xmin, 0.75*xmax, int(0.8*nsamples))
samples_x = np.concatenate(
    [samples_x, rng.uniform(0.75*xmax, xmax, nsamples - int(0.8*nsamples))]
)
samples_x.sort()

samples_y = gt_non_lin_func(samples_x)
samples_y += rng.normal(0, 0.5, len(samples_y))

In [ ]:
non_linreg = NonLinearRegression(
    alphas=np.linspace(0, 3, 10),
    lam=0.5
)
non_linreg.fit(samples_x, samples_y)

xrange = np.array([-0.1, 3.1])
yrange = np.array([-3.1, 2.1])

grid_size = 1000
grid_x, grid_y = np.meshgrid(
    np.linspace(xrange[0] , xrange[1] , grid_size), 
    np.linspace(yrange[0] , yrange[1] , grid_size),
    indexing="xy"
)

non_lin_likelihood = non_linreg.likelihood(grid_x.flatten(), grid_y.flatten())
non_lin_likelihood = non_lin_likelihood.reshape((grid_size, grid_size))

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.scatter(samples_x, samples_y, color="b")
ax.plot(
    np.linspace(xrange[0], xrange[1], 1000),
    non_linreg.predict(np.linspace(xrange[0], xrange[1], 1000)),
    c="r"
)
ax.imshow(
    non_lin_likelihood, 
    extent=xrange.tolist() + yrange.tolist(),  
    origin='lower', 
    aspect='auto',
    cmap="hot"
)
fig.tight_layout()

### Bayesian Non Linear Regression

In [ ]:
gauss_process_reg = GaussProcessRegression(lam=np.sqrt(0.1), sigma_prior=1000)
gauss_process_reg.fit(samples_x, samples_y)

In [ ]:
xrange = np.array([-0.1, 3.1])
yrange = np.array([-3.1, 2.1])

grid_size = 1000
grid_x, grid_y = np.meshgrid(
    np.linspace(xrange[0] , xrange[1] , grid_size), 
    np.linspace(yrange[0] , yrange[1] , grid_size),
    indexing="xy"
)

gauss_process_likelihood = gauss_process_reg.likelihood(
    grid_x.flatten(), grid_y.flatten()
)
gauss_process_likelihood = gauss_process_likelihood.reshape(grid_size, grid_size)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=[15, 7])

ax[0].scatter(samples_x, samples_y, color="b")
ax[0].plot(
    np.linspace(xrange[0], xrange[1], 1000),
    non_linreg.predict(np.linspace(xrange[0], xrange[1], 1000)),
    c="r"
)
ax[0].imshow(
    non_lin_likelihood, 
    extent=xrange.tolist() + yrange.tolist(),  
    origin='lower', 
    aspect='auto',
    cmap="hot"
)
ax[0].set_title("Maximum Likelihood Non Lin")

ax[1].scatter(samples_x, samples_y, c="blue")
ax[1].plot(
    np.linspace(xrange[0], xrange[1], 1000),
    gauss_process_reg.predict(np.linspace(xrange[0], xrange[1], 1000)),
    c="r"
)
ax[1].imshow(
    gauss_process_likelihood, 
    extent=xrange.tolist() + yrange.tolist(),
    origin='lower', 
    aspect='auto',
    cmap="hot"
)
ax[1].set_title("Gaussian Process (Bayes Non Lin)")
fig.tight_layout()

### Sparse Linear Regression

In [ ]:
nsamples = 50
outlier_rate = 0.0
inlier_noise_std = 0.25

direction_3d = np.array([5, 1, 2])
direction = 0.5
offset = 0.75

xmin = 0
xmax = 3.0
xrange = np.array([xmin, xmax])

rng = np.random.default_rng(seed=12345)

samples_x, samples_y = create_noisy_linear(
    direction,
    offset,
    nsamples,
    xrange,
    inlier_noise_std,
    outlier_rate,
    rng
)

samples = np.stack([samples_x, samples_y], axis=0)
values = samples.T @ direction_3d[:-1] + direction_3d[-1] 
values += rng.normal(0, 1.0, nsamples)

In [ ]:
sparse_linreg = SparseLinearRegression(nu=1e-4, thresh=1.0)
sparse_linreg.fit(samples, values)

In [ ]:
xrange = np.array([0, 3])
yrange = np.array([0, 3])

grid_size = 1000
grid_x, grid_y = np.meshgrid(
    np.linspace(xrange[0], xrange[1], 1000),
    np.linspace(yrange[0], yrange[1], 1000),
    indexing="xy"
)
grid = np.stack([grid_x.flatten(), grid_y.flatten()], axis=0)

groundtruth_values = grid.T @ direction_3d[:-1] + direction_3d[-1]
groundtruth_values = groundtruth_values.reshape((grid_size, grid_size))

predicted_values = sparse_linreg.predict(grid)
predicted_values = predicted_values.reshape((grid_size, grid_size))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=[15, 7])

vmin=min(groundtruth_values.min(), values.min())
vmax=max(groundtruth_values.max(), values.max())
ax[0].imshow(
    groundtruth_values, 
    cmap="hot",
    extent=xrange.tolist() + yrange.tolist(),
    vmin=vmin,
    vmax=vmax
)
ax[0].scatter(
    samples_x, 
    samples_y, 
    c=values, 
    cmap='hot', 
    edgecolors='black', 
    linewidths=1,
    vmin=vmin,
    vmax=vmax
)
ax[0].set_title("Ground truth value")

vmin=min(predicted_values.min(), values.min())
vmax=max(predicted_values.max(), values.max())
ax[1].imshow(
    predicted_values, 
    cmap="hot",
    extent=xrange.tolist() + yrange.tolist(),
    vmin=vmin,
    vmax=vmax
)
ax[1].scatter(
    samples_x, 
    samples_y, 
    c=values, 
    cmap='hot', 
    edgecolors='black', 
    linewidths=1,
    vmin=vmin,
    vmax=vmax
)
ax[1].set_title("Predicted value with sparse model")
fig.tight_layout()

### Relevance Vector Non Linear

In [ ]:
gt_non_lin_func = lambda x: 1.5 * np.sin(x ** 2) + np.cos(x ** 3)

xmin, xmax = 0, 3.05
xrange = np.linspace(xmin, xmax, 1000)

rng = np.random.default_rng(seed=12345)
nsamples = 50
samples_x = rng.uniform(xmin, 0.75*xmax, int(0.8*nsamples))
samples_x = np.concatenate(
    [samples_x, rng.uniform(0.75*xmax, xmax, nsamples - int(0.8*nsamples))]
)
samples_x.sort()

samples_y = gt_non_lin_func(samples_x)
samples_y += rng.normal(0, 0.5, len(samples_y))

In [ ]:
relevance_vector_reg = RelevanceVectorRegression(
    nu = 1e-7,
    lam = 0.3,
    thresh=1000
)
relevance_vector_reg.fit(samples_x, samples_y)

In [ ]:
xrange = np.array([-0.1, 3.1])
yrange = np.array([-4.0, 3.0])

grid_size = 1000
grid_x, grid_y = np.meshgrid(
    np.linspace(xrange[0] , xrange[1] , grid_size),
    np.linspace(yrange[0] , yrange[1] , grid_size),
    indexing="xy"
)

grid_x = grid_x.flatten()
grid_y = grid_y.flatten()

relevance_vector_likelihood = relevance_vector_reg.likelihood(grid_x, grid_y)
relevance_vector_likelihood = relevance_vector_likelihood.reshape((grid_size, grid_size))

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.imshow(
    relevance_vector_likelihood,
    cmap="hot",
    extent=[xrange[0], xrange[1], yrange[0], yrange[1]],
    origin='lower',
    aspect='auto',
)
ax.plot(
    np.linspace(xrange[0], xrange[1], 1000),
    relevance_vector_reg.predict(np.linspace(xrange[0], xrange[1], 1000)),
    c="r"
)
ax.scatter(
    samples_x[relevance_vector_reg.tokeep], 
    samples_y[relevance_vector_reg.tokeep], 
    s=70, 
    c="green", 
    edgecolors="w"
)
ax.scatter(
    samples_x[~relevance_vector_reg.tokeep], 
    samples_y[~relevance_vector_reg.tokeep],
    s=2, 
    c="blue"
)
fig.tight_layout()